In [107]:
from __future__ import absolute_import, division
from __future__ import print_function
import autograd.numpy as np
import autograd.numpy.random as npr
from autograd.scipy.misc import logsumexp
from autograd import grad
from autograd.misc.flatten import flatten
from autograd.misc.optimizers import adam
from numpy import genfromtxt
from math import pi
from operator import add
import matplotlib.pyplot as plt


## Première partie : prédiction de latitude et longitude

In [82]:
#création des données
my_data = genfromtxt('default_plus_chromatic_features_1059_tracks.txt', delimiter=',')
train = my_data[1:799,]
test = my_data[800:,]

### Code

In [83]:
#Distance du Grand Cercle
def dgc(lat_1, lat_2, lon_1, lon_2) :
    #conversion degré -> rad
    lat_1, lat_2, lon_1, lon_2 = (pi/180)* lat_1, (pi/180)*lat_2, (pi/180)*lon_1, (pi/180)*lon_2
    q = np.sin((lat_1-lat_2)/2)**2+np.cos(lat_1)*np.cos(lat_2)*np.sin((lon_1-lon_2)/2)**2
    return(2*6378*np.arcsin(np.sign(q)*np.sqrt(np.abs(q))))

print("distance entre les deux pôles...")
print("...par la distance du grand cercle")
print(dgc(90,-90, 0, 0))
print("... par la formule circonférence = 2 pi R")
print(np.pi*6378)
print("")

print("distance entre deux points de l'équateur diamétralement opposés...")
print("...par la distance du grand cercle")
print(dgc(0,0, 0, 180))
print("... par la formule circonférence = 2 pi R")
print(np.pi*6378)
print("")

print("distance Paris Bogota...")
print("...par la distance du grand cercle")
print(dgc(4.4,48.5, -74, 2.2))
print("... trouvée sur Internet")
print(8633)




distance entre les deux pôles...
...par la distance du grand cercle
20037.0779446
... par la formule circonférence = 2 pi R
20037.0779445957

distance entre deux points de l'équateur diamétralement opposés...
...par la distance du grand cercle
20037.0779446
... par la formule circonférence = 2 pi R
20037.0779445957

distance Paris Bogota...
...par la distance du grand cercle
8636.14950817
... trouvée sur Internet
8633


In [117]:
#Fonction de création des paramètres directement inspirée de l'exemple
def init_random_params(scale, layer_sizes, rs=npr.RandomState(0)):
    """Build a list of (weights, biases) tuples, one for each layer."""
    return [(rs.randn(insize, outsize) * scale,   # weight matrix
             rs.randn(outsize) * scale)           # bias vector
            for insize, outsize in zip(layer_sizes[:-1], layer_sizes[1:])]

#Fonction de prédiction directement inspirée de l'exemple
def nn_predict(params, inputs, nonlinearity = np.tanh):
    for W, b in params:
        outputs = np.dot(inputs, W) + b
        inputs = nonlinearity(outputs)
    output = nonlinearity(outputs)*np.array([90, 180])
    return outputs


init_scale = 0.1
weight_prior_variance = 10.0
init_params = init_random_params(init_scale, layer_sizes=[116, 800, 800, 2])

#fonction objectif faisant intervenir le grand cercle
def objective(params, t) : 
    t = t%np.shape(train)[0]
    predict = nn_predict(params, train[t,0:116] , nonlinearity=np.tanh)
    return(dgc(predict[0], train[t,116], predict[1], train[t,117])**2)


### Résultats

In [ ]:
#Optimisation 
optimized_params = adam(grad(objective), init_params, step_size=0.01, num_iters=10000)

C:\Users\Utilisateur\Anaconda3\lib\site-packages\autograd\numpy\numpy_vjps.py:88: RuntimeWarning: overflow encountered in square
  defvjp(anp.tanh,   lambda ans, x : lambda g: g / anp.cosh(x) **2)


In [ ]:
#Prédiction et retour d'erreur
def pred (optimized_params) :
    preds = []
    for i in range(np.shape(test)[0]) : 
        preds.append(nn_predict(optimized_params, inputs=test[i, 0:116]))
    return(np.array(preds))
#retourne un array de la différence de latitude et de longitude entre prédiction et référence
def  errors (optimized_params) : 
    preds = pred(optimized_params)
    true = []
    for i in range(np.shape(test)[0]) : 
         true.append(test[i, 116:])
    true = np.array(true)
    return(preds-true)
errors (optimized_params)
plt.scatter(errors (optimized_params).transpose()[0,],errors (optimized_params).transpose()[1,])
plt.show()

## Seconde partie : utilisation du heatmap

In [12]:
#création des données
heatmap = np.loadtxt('heatmap.txt')
my_data = genfromtxt('default_plus_chromatic_features_1059_tracks.txt', delimiter=',')
my_data = my_data[0:len(my_data), 0:116]
my_data = np.concatenate((my_data, heatmap), 1)
train = my_data[np.random.random_integers(1, high=799, size=10000),]
test = my_data[800:,]

In [49]:
init_scale = 0.1
weight_prior_variance = 10.0
init_params = init_random_params(init_scale, layer_sizes=[116, 200, 200, 400])


#Fonction de prédiction directement inspirée de l'exemple
#différence par rapport à avant : on fait en sorte que les outputs soient bien compris entre 0 et 1
def nn_predict(params, inputs, nonlinearity = np.tanh):
    for W, b in params:
        outputs = np.dot(inputs, W) + b
        inputs = nonlinearity(outputs)
    outputs =( np.tanh(outputs)+1)/2
    return outputs

# Divergence de Kullbacl-Leibler : on ajoute un epsilon aux histogrammes pour éviter les divisions par 0
def kullback_leibler(params, t) : 
    predict = nn_predict(params, train[t,0:116] , nonlinearity=np.tanh)
    predict = predict +0.000001
    return(np.sum(predict*np.log(predict/(train[t,116:]+0.000001))))



In [50]:
kullback_leibler(init_params, 1)

5031.6572941207578